# 0.1. imports

In [4]:
# lca
import bw2data
import bw2io
# i/o
import gzip
# data science
import pandas as pd
import  numpy
import scipy

In [2]:
%load_ext autoreload
%autoreload 2

# 1. load required data
## 1.1. load `pylcaio` object

In [5]:
with gzip.open('/Users/michaelweinold/data/data_processed/hybrid.pickle','rb') as f:
    hybrid_pickle = pd.read_pickle(f)

# 2. refactoring
## 2.1. legacy function

In [ ]:
def import_exiobase_into_brightway2(self):

        print('Starting import of exiobase')
        print('Aggregating results')
        # calculating aggregated results for exiobase
        self.Lio = scipy.sparse.csr_matrix(
            np.linalg.solve(np.eye(self.A_io.shape[0]) - self.A_io.todense(), np.eye(self.A_io.shape[0])))
        aggregated_results = pd.DataFrame((self.C_io.dot(self.F_io).dot(self.Lio)).todense(), self.impact_categories_IO,
                                          pd.MultiIndex.from_product([self.regions_of_IO, self.sectors_of_IO]))
        print('Formating data')
        # create hash codes for exiobase sectors
        give_hash_to_exio = dict.fromkeys(aggregated_results.columns)
        give_hash_to_exio = {k: hashlib.md5(str(k).encode()).hexdigest() for k, v in give_hash_to_exio.items()}
        aggregated_results.columns = [give_hash_to_exio[i] for i in aggregated_results.columns]

        # link IMPACT World+ impact categories typology to the methods abbreviations used in brightway2
        IW_pylcaio_to_bw2 = dict.fromkeys(self.impact_categories_IO)
        for IW_category in IW_pylcaio_to_bw2:
            if 'PDF' in IW_category or 'DALY' in IW_category:
                IW_pylcaio_to_bw2[IW_category] = \
                bw2data.methods.get(('IMPACTWorld+ (Default_Recommended_Damage 1.46)', IW_category.split(' (')[0]))[
                    'abbreviation']
            else:
                IW_pylcaio_to_bw2[IW_category] = \
                bw2data.methods.get(('IMPACTWorld+ (Default_Recommended_Midpoint 1.28)', IW_category.split(' (')[0]))[
                    'abbreviation']
        aggregated_results.index = list(IW_pylcaio_to_bw2.values())

## 2.2. investigation

## 2.3. new function
